# **This notebook sets up a project that is used in the paper:**
*"Integrating emerging technologies deployed at scale within prospective life-cycle assessment"*, Charalambous et al. (2024)

### **The instructions below should be followed for the <font color="blue">non-integrated LCA approach</font>**

##### **In this notebook we will do the following:** 
- We create a new Brightway project and set it up with biosphere and ecoinvent 3.8
- We install the premise databases that we need using the external scenario that we created as *<font color="blue">brightway databases</font>*
- We create new databases that include the transport activities we want to use
- We install the LCIA methods that we need

**Execute all the cells and the project will be ready to run calculations**

*Author: Margarita Charalambous*

In [2]:
from brightway2 import *
import bw2data as bd
import pyprind
from pathlib import Path
import pandas as pd
import brightway2 as bw
from bw2io import ExcelLCIAImporter
from premise import *

### **1. Set up project**

In [3]:
projects.set_current("Integrated LCA project") #<---- Project name

##### - **Set up biosphere**

In [3]:
if 'biosphere3' in databases:
    print('Biosphere database already installed')
else:
    bw2setup()

Biosphere database already installed


##### - **Install Ecoinvent**

In [4]:
ECOINVENT_DIR = r"Y:/projects/5- Prospective LCA/data/external/ecoinvent 3.8_cutoff_ecoSpold02/datasets" # <- insert the path to ecoinvent 3.8 here
ECOINVENT_NAME = 'ecoinvent 3.8 cutoff' # <- Same name as in premise installation you will use later

if 'ecoinvent 3.8 cutoff' in databases:
    print("Database has already been imported.")
else:
    ei38cut = SingleOutputEcospold2Importer(ECOINVENT_DIR, ECOINVENT_NAME, use_mp=True)
    ei38cut.apply_strategies()
    ei38cut.statistics()
    ei38cut.write_database()

Extracting XML data from 19565 datasets
Extracted 19565 datasets in 133.90 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy:

Writing activities to SQLite3 database:


19565 datasets
629959 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


Title: Writing activities to SQLite3 database:
  Started: 08/29/2024 17:48:47
  Finished: 08/29/2024 17:49:23
  Total time elapsed: 00:00:35
  CPU %: 100.00
  Memory %: 11.06
Created database: ecoinvent 3.8 cutoff


##### - **Install premise with HDdiesel external scenario**

- This is the link to the repository that hosts the user-defined-scenario: https://github.com/MargotCha/HDdiesel-prospective-scenarios

In [7]:
projects.set_current("Integrated LCA project")

from premise import *
from datapackage import Package
clear_cache()

PATH_TO_FILE = r"PATH_TO_FILE" # --> Add path to the directory where the datapackage file from the external scenario
fp = r"PATH_TO_FILE/HDfuels-prospective-scenarios/datapackage.json" 
synfuels = Package(fp)
scenarios = [    
    {"model": "remind", "pathway":"SSP2-Base", "year": 2020},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2025},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2030},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2035},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2040},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2045},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2050},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2020},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2025},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2030},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2035},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2040},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2045},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2050},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2020},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2025},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2030},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2035},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2040},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2045},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2050}]
ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', # <- This is the key provided by the premise maintainers
        external_scenarios=[synfuels]
)

ndb.update_trucks() 
ndb.update_electricity()
ndb.update_dac()
ndb.update_fuels()
ndb.update_external_scenario()

ndb.write_db_to_brightway()

Cache folder cleared!
premise v.(1, 5, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------

100%|██████████| 19565/19565 [00:02<00:00, 9234.11it/s]


Adding exchange data to activities


100%|██████████| 629959/629959 [00:35<00:00, 17780.75it/s]


Filling out exchange data


...

Title: Writing activities to SQLite3 database:
  Started: 08/29/2024 23:54:36
  Finished: 08/29/2024 23:55:20
  Total time elapsed: 00:00:43
  CPU %: 99.00
  Memory %: 36.17
Created database: ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2040
Prepare database 20.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 08/30/2024 00:09:39
  Finished: 08/30/2024 00:10:22
  Total time elapsed: 00:00:43
  CPU %: 99.20
  Memory %: 34.21
Created database: ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2045
Prepare database 21.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:49


Title: Writing activities to SQLite3 database:
  Started: 08/30/2024 00:25:10
  Finished: 08/30/2024 00:26:00
  Total time elapsed: 00:00:49
  CPU %: 99.40
  Memory %: 33.61
Created database: ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2050
Generate scenario report.
Report saved under c:\Users\mcharalam\dev\prospectiveLCA\Premise\pLCA\Integrated-LCA-master-main\Codes\Setting up\export\scenario_report.
Generate change report.
Report saved under c:\Users\mcharalam\dev\prospectiveLCA\Premise\pLCA\Integrated-LCA-master-main\Codes\Setting up.


### **2. Create databases to run calculations**

#### **A. Create new empty databases**

In [4]:
db_store = []
for i in databases:
    if "ecoinvent_cutoff_3.8_remind" in i: 
        db_store.append(i)
db_store

['ecoinvent_cutoff_3.8_remind_SSP2-Base_2020',
 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2025',
 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2030',
 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2035',
 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2040',
 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2045',
 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2050',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2020',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2025',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2030',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2035',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2040',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2045',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg1150_2050',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2020',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2025',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2030',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2035',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBudg500_2040',
 'ecoinvent_cutoff_3.8_remind_SSP2-PkBud

In [10]:
for db in db_store:
    if bd.Database(db.split('_3.8_')[1]) in databases:
        print(f'Database already exists')
    else:
        newDB = bd.Database(db.split('_3.8_')[1])
        newDB.register()

#### **B. Add the activities you want to use in each database**

- Find activities to add in each database

In [11]:
acts_get =['transport, freight, lorry, diesel, 40t gross weight,']
acts_get

['transport, freight, lorry, diesel, 40t gross weight,']

- Add the transport activities to the new databases 

In [12]:
import uuid

In [13]:
#This copies the diesel truck activities from the prospective databases from all the REMIND regions

for db in pyprind.prog_percent(db_store): 
    get_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    for i in acts_get:
        get_activity = [act for act in Database(db) if i in act["name"] and "long haul" in act["name"] and db.split("_")[-1] in act["name"] and not act["location"]=="RER"]
        for act in get_activity:
            act.copy(code=uuid.uuid4().hex, name = act["name"] + f'_fixedbckg', database=get_db.name) 

[100 %] Time elapsed: 00:05:21 | ETA: 00:00:00
Total time elapsed: 00:05:21


#### **C. Duplicate diesel truck activities already present in the new databases and rename them to reflect the two different synthetic diesel fuels investigated in the main manuscript**

In [14]:
#Consider only taking diesel trucks that are "World" location to replicate the results of the main manuscript. If you want to use all the REMIND regions just do not specify a location in the code below

for db in pyprind.prog_percent(db_store): 
    get_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    diesel_truck = [act for act in get_db if 'transport, freight, lorry, diesel, 40t gross weight,' in act['name'] and 'long haul' in act['name'] and db.split("_")[-1] in act["name"] and act["location"]=="World"]
    diesel_prod1 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'PEM' in act['name'] and '>3MW' in act['name']][0]
    diesel_prod2 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'wet poplar' in act['name'] and 'CCS' in act['name']][0]
    diesel = [diesel_prod1, diesel_prod2]

    for i in diesel:
        if 'wind' in i['name'] and 'DAC' in i['name']:
            nameco2 = i['name'].split('--')[1].split(' ')[-1]
            nameh2 = i['name'].split('--')[2].split(',')[1].split(' ')[-1]
            for act in diesel_truck:
                act.copy(code=uuid.uuid4().hex, name = act['name'] + f' ' + nameco2+ f'_' +nameh2, database=get_db.name) 
        elif 'biomass' in i['name'] and 'DAC' in i['name']:
            nameco2 = i['name'].split('--')[1].split(' ')[-1]
            nameh2 = i['name'].split('--')[2].split(',')[1].split(' ')[3]
            for act in diesel_truck:
                act.copy(code=uuid.uuid4().hex, name = act['name'] + f' ' + nameco2 + f'_'  + nameh2, database=get_db.name) 

[100 %] Time elapsed: 00:02:20 | ETA: 00:00:00
Total time elapsed: 00:02:20


#### **D. Removing the market of diesel that is fueling the trucks**

In [15]:
for db in pyprind.prog_percent(db_store): 
    new_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    old_db = [bd.Database(db)][0]
    diesel_prod1 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'PEM' in act['name'] and '>3MW' in act['name'] and not "GLO" in act["location"]]
    diesel_prod3 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'wet poplar' in act['name'] and 'CCS' in act['name'] and not "GLO" in act["location"]]
    
    for act in new_db: 
        for i in diesel_prod1: #<----- DAC/Wind
            if "diesel" in act['name'] and "DAC" in act['name'] and "wind" in act['name'] and act['location']==i['location']:
                for exc in act.technosphere(): 
                    if "market for diesel, low-sulfur" in exc.input['name']:
                        get_amount = exc['amount']
                        act.new_exchange(input=i.key,amount=get_amount,unit=i['unit'],type='technosphere').save()
                        exc.delete() 
    for act in new_db: 
        for i in diesel_prod3: #<----- DAC/poplar
            if "diesel" in act['name'] and "DAC" in act['name'] and "poplar" in act['name'] and act['location']==i['location']:
                for exc in act.technosphere(): 
                    if "market for diesel, low-sulfur" in exc.input['name']:
                        get_amount = exc['amount']
                        act.new_exchange(input=i.key,amount=get_amount,unit=i['unit'],type='technosphere').save()
                        exc.delete() 

[100 %] Time elapsed: 00:01:41 | ETA: 00:00:00
Total time elapsed: 00:01:41


#### **E. Creating world diesel market for each synthetic fuel**

- Each synthetic global market should take the fractions of the global market of diesel as in the activity

#### 1. Add the diesel market in each database and rename it based on the synthetic diesel fuel

In [16]:
for db in pyprind.prog_percent(db_store): 
    get_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    act = [act for act in bd.Database(db) if act['name'] =='market for diesel, low-sulfur' and act['location']=='World'][0]
    diesel_prod1 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'PEM' in act['name'] and '>3MW' in act['name']][0]
    diesel_prod2 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'wet poplar' in act['name'] and 'CCS' in act['name']][0]
    diesel = [diesel_prod1, diesel_prod2]

    for i in diesel:
        if 'wind' in i['name'] and 'DAC' in i['name']:
            nameco2 = i['name'].split('--')[1].split(' ')[-1]
            nameh2 = i['name'].split('--')[2].split(',')[1].split(' ')[-1]
            act.copy(code=uuid.uuid4().hex, name = act["name"] + f'_fixedbckg'+ f' ' + nameco2+ f'_' +nameh2, database=get_db.name)   
        elif 'biomass' in i['name'] and 'DAC' in i['name']:
            nameco2 = i['name'].split('--')[1].split(' ')[-1]
            nameh2 = i['name'].split('--')[2].split(',')[1].split(' ')[3]
            act.copy(code=uuid.uuid4().hex, name = act['name'] + f'_fixedbckg'+ f' ' + nameco2+ f'_' +nameh2, database=get_db.name) 



[100 %] Time elapsed: 00:02:51 | ETA: 00:00:00
Total time elapsed: 00:02:51


#### 2.Change exchanges in diesel markets

In [5]:
for db in pyprind.prog_percent(db_store):     
    new_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    old_db = [bd.Database(db)][0]
    diesel_prod1 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'PEM' in act['name'] and '>3MW' in act['name']]
    diesel_prod2 = [act for act in bd.Database(db) if 'diesel production,--' in act['name'] and 'DAC' in act['name'] and 'wet poplar' in act['name'] and 'CCS' in act['name']]

    for i in diesel_prod1: #<----- DAC/Wind
        for act in new_db:
            if "market for diesel" in act['name'] and "DAC" in act['name'] and "wind" in act['name']:
                for exc in act.technosphere(): 
                    if exc.input['location'] == i['location'] :
                        get_amount = exc['amount']
                        act.new_exchange(input=i.key,amount=get_amount,unit=i['unit'],type='technosphere').save()
                        exc.delete() 
    for i in diesel_prod2: #<----- DAC/poplar
        for act in new_db: 
            if "market for diesel" in act['name'] and "DAC" in act['name'] and "poplar" in act['name']:
                for exc in act.technosphere(): 
                    if exc.input['location'] == i['location'] :
                        get_amount = exc['amount']
                        act.new_exchange(input=i.key,amount=get_amount,unit=i['unit'],type='technosphere').save()
                        exc.delete() 


[100 %] Time elapsed: 00:01:39 | ETA: 00:00:00
Total time elapsed: 00:01:39


#### **F. Adding world fuel markets to world transport activities**

In [6]:
for db in pyprind.prog_percent(db_store):     
    new_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    old_db = [bd.Database(db)][0]
    diesel_truck = [act for act in new_db if 'transport, freight, lorry, diesel, 40t gross weight,' in act['name'] 
                        and 'long haul' in act['name'] and db.split("_")[-1] in act["name"] and act["location"]=="World"]
        
    for i in diesel_truck: 
        if "DAC" in i['name'] and "wind" in i['name']:#<----- DAC/Wind
            for act in new_db:
                if "market for diesel" in act['name'] and "DAC" in act['name'] and "wind" in act['name']:
                    for exc in i.technosphere(): 
                        if 'market for diesel, low-sulfur' in exc.input['name']:
                            get_amount = exc['amount']
                            i.new_exchange(input=act.key,amount=get_amount,unit=act['unit'],type='technosphere').save()
                            exc.delete() 

        if "DAC" in i['name'] and "poplar" in i['name']:#<----- DAC/poplar
            for act in new_db:
                if "market for diesel" in act['name'] and "DAC" in act['name'] and "poplar" in act['name']:
                    for exc in i.technosphere(): 
                        if 'market for diesel, low-sulfur' in exc.input['name']:
                            get_amount = exc['amount']
                            i.new_exchange(input=act.key,amount=get_amount,unit=act['unit'],type='technosphere').save()
                            exc.delete() 


[  4 %] Time elapsed: 00:00:00 | ETA: 00:00:00

[100 %] Time elapsed: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


### **2. Install LCIA methods**

#### Create/update LCIA methods

<font color="grey">*The following information is taken from the repository of Robert Istrate: https://github.com/robyistrate/internet-environmental-footprint*</font>

Here we implement some new methods that are not available in the biosphere database compatible with Ecoinvent 3.8, including:

- IPCC 2021 global warming potentials (GWPs) adapted to premise

The following data files are required (to be obtained from the corresponding sources and placed in the data container):

- Methods implementation in Ecoinvent 3.9: "LCIA Implementation 3.9.1.xlsx" (available at https://ecoinvent.org/)
- Because the name of some elementary flows change from Ecoinvent 3.8 to 3.9.1, the change report file is also required: "Change Report Annex v3.8 - v3.9.xlsx" (available at https://ecoinvent.org/)
- 100-year GWPs adapted to premise: "lcia_gwp_100a_w_bio.xlsx" (available at https://github.com/polca/premise_gwp)

**A. IPCC 2013 with hydrogen and bioCO2 flows**

In [7]:
from premise_gwp import add_premise_gwp
add_premise_gwp() 

Adding ('IPCC 2013', 'climate change', 'GWP 20a, incl. H')
Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.12 seconds
Wrote 1 LCIA methods with 216 characterization factors
Adding ('IPCC 2013', 'climate change', 'GWP 100a, incl. H')
Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.12 seconds
Wrote 1 LCIA methods with 216 char

In [8]:
# Import LCIA implementation data
PATHTOFILE = r"ADD_PATH_TO_FILE_HERE" # <- Path to the directory where the LCIA implementation files are stored
METHODS_DIR = Path(r"PATHTOFILE\Data\LCIA\LCIA Implementation 3.9.1.xlsx") 
CHANGE_REPORT_DIR = Path(r'PATHTOFILE\Data\LCIA\Change Report Annex v3.8 - v3.9.xlsx')
PREMISE_GWP_DIR = Path(r'PATHTOFILE\Data\LCIA\lcia_gwp2021_100a_w_bio.xlsx')

lcia_ei_391 = pd.read_excel(METHODS_DIR, sheet_name='CFs')
lcia_change_ei39vs38 = pd.read_excel(CHANGE_REPORT_DIR, sheet_name='LCIA Changes')
premise_gwp = pd.read_excel(PREMISE_GWP_DIR)

In [9]:
# Map biosphere flows
map_bio_keys = {}
for ef in Database('biosphere3'):
    map_bio_keys[(ef['name'], ef['categories'])] = ef.key

**B. Climate change: 100-year GWPs from IPCC 2021 adapted to premise**

Since prospective databases generated with *premise* may involve inventories containing
net negative emission technologies (NETs), it is highly recommended to account for
biogenic CO2 flows as well as CO2 captured from the atmosphere. For further details see: https://github.com/polca/premise_gwp

Premise GWPs are based on IPCC2013. Here we update the method with GWPs from IPCC2021

In [11]:
# Get 100-year GWPs from IPCC 2021 as implemented in ecoinvent 3.9.1
ipcc2021_gwp_100a = lcia_ei_391[(lcia_ei_391['Method'] == 'IPCC 2021') & 
                                (lcia_ei_391['Category'] == 'climate change: including SLCFs') & 
                                (lcia_ei_391['Indicator'] == 'global warming potential (GWP100)')]

# Adapt GWPs from IPCC 2021 to premise GWPs:
premise_ipcc2021_gwp_100a = dict()
for index, row in premise_gwp.iterrows():
    premise_ipcc2021_gwp_100a.update({(row['name'], row['categories']): row['amount']})

    # Update existing and add new characterization factors based on IPCC2021
    for index, row in ipcc2021_gwp_100a.iterrows():
        ef_info = (row['Name'], row['Compartment'] + '::' + row['Subcompartment'])
        # Update existing elementary flows:
        if ef_info in premise_ipcc2021_gwp_100a:
            premise_ipcc2021_gwp_100a[ef_info] = row['CF']
        else:
           premise_ipcc2021_gwp_100a.update({ef_info: row['CF']})

    # Export the modified IPCC method:
    premise_ipcc2021_df = pd.DataFrame(premise_ipcc2021_gwp_100a, index=['amount']).T.reset_index()
    premise_ipcc2021_df.columns = ['name', 'categories', 'amount']
    premise_ipcc2021_df.to_excel(Path(r"PATHTOFILE\Data\LCIA\LCIA_gwp_100a_ipcc2021_premise.xlsx"), index=False)

In [13]:
# Write method:
metadata =(
           ("IPCC 2021", "climate change: including SLCFs", "GWP 100a, incl. H and bio CO2"),
            "kg CO2-Eq",
            "IPCC 20121, with CFs for hydrogen and biogenic CO2 flows",
            Path(r"PATHTOFILE\Data\LCIA\LCIA_gwp_100a_ipcc2021_premise.xlsx"),
          )

ipcc2021_method = ExcelLCIAImporter(filepath=metadata[-1], name=metadata[0], unit=metadata[1], description=metadata[2])
ipcc2021_method.apply_strategies()
ipcc2021_method.drop_unlinked()
assert len(list(ipcc2021_method.unlinked)) == 0
ipcc2021_method.write_methods(overwrite=True, verbose=True)

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.13 seconds
Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
Wrote 1 LCIA methods with 245 characterization factors
